In [1]:
import imutils
import dlib
import cv2
import copy
import math
import numpy as np


In [2]:

def calculateFingers(res,drawing):  # -> finished bool, cnt: finger count
    #  convexity defect
    hull = cv2.convexHull(res, returnPoints=False)
    if len(hull) > 3:
        defects = cv2.convexityDefects(res, hull)
        if type(defects) != type(None):  # avoid crashing.   (BUG not found)

            cnt = 0
            for i in range(defects.shape[0]):  # calculate the angle
                s, e, f, d = defects[i][0]
                start = tuple(res[s][0])
                end = tuple(res[e][0])
                far = tuple(res[f][0])
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem
                if angle <= math.pi / 2:  # angle less than 90 degree, treat as fingers
                    cnt += 1
                    cv2.circle(drawing, far, 8, [211, 84, 0], -1)
            return True, cnt
    return False, 0

In [3]:
# Now let's use the detector as you would in a normal application.  First we
# will load it from disk.
detector = dlib.simple_object_detector("detector_hand.svm")


In [13]:
image = cv2.imread("dataset\\images.jpg")
image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [14]:
cv2.imshow('ori',gray)
k = cv2.waitKey(0)

In [15]:
rects = detector(gray)

In [16]:
for k, d in enumerate(rects):
    print(k,d)

0 [(257, 161) (472, 376)]


In [17]:
for k, d in enumerate(rects):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))

Detection 0: Left: 257 Top: 161 Right: 472 Bottom: 376


In [18]:
roi = gray[d.top():d.bottom(), d.left():d.right()]

In [19]:
ret, thresh_fin = cv2.threshold(roi, 150, 255, cv2.THRESH_BINARY)

for i in range(170, 190, 20):
    ret, thresh = cv2.threshold(roi, i, 255, cv2.THRESH_BINARY)
    thresh_fin += thresh
mb = cv2.medianBlur(thresh_fin,3)
cv2.imshow('ori', mb)


# get the coutours
thresh1 = copy.deepcopy(mb)
im2, contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
length = len(contours)
maxArea = -1
if length > 0:
    for i in range(length):  # find the biggest contour (according to area)
        temp = contours[i]
        area = cv2.contourArea(temp)
        if area > maxArea:
            maxArea = area
            ci = i

    res = contours[ci]
    res = res + [d.left(),d.top()]
    hull = cv2.convexHull(res)
    
    drawing = image
    cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)
    cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)
    
    drawing1 = np.zeros(image.shape, np.uint8)
    cv2.drawContours(drawing1, [res], 0, (0, 255, 0), 2)
    cv2.drawContours(drawing1, [hull], 0, (0, 0, 255), 3)

cv2.imshow('contour-ori', drawing)
cv2.imshow('contour-black', drawing1)
#    isFinishCal,cnt = calculateFingers(res,drawing)
#    if isFinishCal is True and cnt <= 2:
#        print(cnt)
            #app('System Events').keystroke(' ')  # simulate pressing blank space

k = cv2.waitKey(0)